In [1]:
import os
import random
import torch
import json

import torchvision.transforms as T
from tqdm.auto import tqdm

In [2]:
from facade_project import FACADE_LABELME_ORIGINAL_DIR, FACADE_ROT_IMAGES_TENSORS_DIR, FACADE_ROT_HEATMAPS_TENSORS_DIR, FACADE_ROT_HEATMAPS_INFOS_PATH, \
    NUM_IMAGES, NUM_ROTATIONS, LABEL_NAME_TO_VALUE, IMG_MAX_SIZE, CUT_MARGIN
from facade_project.utils.load import load_tuple_from_json
from facade_project.geometry.heatmap import extract_heatmaps_info
from facade_project.geometry.masks import crop_pil, get_bbox
from facade_project.geometry.image import resize, rotate
from facade_project.show import *
from facade_project.geometry.heatmap import build_heatmaps, \
    rotate as rotate_info,\
    crop as crop_info,\
    resize as resize_info

In [3]:
img_paths = [os.path.join(FACADE_LABELME_ORIGINAL_DIR, fname) for fname in sorted(os.listdir(FACADE_LABELME_ORIGINAL_DIR))]

len(img_paths)

418

In [4]:
def get_heatmaps_filename(labels, idx, jdx):
    return '{}/heatmaps_{}_{:03d}_{:03d}.torch'.format(FACADE_ROT_HEATMAPS_TENSORS_DIR, '-'.join(sorted(labels)), idx, jdx)    

In [5]:
def get_img_filename(prefix, idx, jdx):
    return '{}/{}_{:03d}_{:03d}.torch'.format(FACADE_ROT_IMAGES_TENSORS_DIR, prefix, idx, jdx)

# Generate
Only run this to generate the data

In [ ]:
# in case we want to add more rotations
rotation_offset = 0
num_rot = NUM_ROTATIONS
max_size = IMG_MAX_SIZE
heatmap_infos = dict()

for idx, path in enumerate(tqdm(img_paths)):
    heatmap_info = extract_heatmaps_info(json.load(open(path, mode='r')))

    img_original, lbl_original = load_tuple_from_json(path)
    img_pil, lbl_pil = T.ToPILImage()(img_original), T.ToPILImage()(lbl_original)

    if idx not in heatmap_infos:
        heatmap_infos[idx] = dict()

    for jdx in range(num_rot):

        info = heatmap_info

        angle = random.randint(1, 10)
        if random.randint(0,1) == 0:
            angle *= -1
        info = rotate_info(info, angle)
        img = rotate(img_pil, angle, itp_name='BI')
        lbl = rotate(lbl_pil, angle, itp_name='NN')

        # CROP - CUT BORDERS
        bbox = get_bbox(lbl, cut_margin=CUT_MARGIN)
        tl_x, tl_y, br_x, br_y = bbox
        bbox_w, bbox_h = br_x - tl_x, br_y - tl_y
        if bbox_w > bbox_h:
            dif = min(bbox_w - bbox_h, CUT_MARGIN)
            bbox = tl_x, max(0, tl_y - dif//2), br_x, min(br_y + dif//2, img.size[1])
        else:
            dif = min(bbox_h - bbox_w, CUT_MARGIN)
            bbox = max(0, tl_x - dif//2), tl_y, min(br_x + dif//2, img.size[0]), br_y

        img = crop_pil(img, bbox)
        lbl = crop_pil(lbl, bbox)
        info = crop_info(info, bbox)

        # RANDOM CROP AND RESIZE
        w, h = img.size
        crop_size = int(min(w, h) * (1 - random.random() / 3))
        top = random.randint(0, h - crop_size)
        left = random.randint(0, w - crop_size)
        bbox2 = (left, top, left + crop_size, top + crop_size)
        img = crop_pil(img, bbox2)
        lbl = crop_pil(lbl, bbox2)
        info = crop_info(info, bbox2)

        resize_size = (IMG_MAX_SIZE, IMG_MAX_SIZE)
        img = resize(img, size=resize_size, itp_name='BI')
        lbl = resize(lbl, size=resize_size, itp_name='NN')
        info = resize_info(info, resize_size)

        img = T.ToTensor()(img)
        lbl = (T.ToTensor()(lbl) * 255).int()  
        heatmap_infos[idx][jdx] = info

        json.dump(heatmap_infos, open(FACADE_ROT_HEATMAPS_INFOS_PATH, mode='w'))

        torch.save(img, get_img_filename('img', idx, jdx + rotation_offset))
        torch.save(lbl, get_img_filename('lbl', idx, jdx + rotation_offset))

        labels=['window', 'door']
        hmaps = build_heatmaps(info, labels, heatmap_types=['center', 'width', 'height'])
        torch.save(hmaps, get_heatmaps_filename(labels, idx, jdx + rotation_offset))


In [ ]:
1+1